#### Importation des librairies

In [1]:
import pandas as pd
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import lightgbm as lgbm
import numpy as np
from sklearn.model_selection import KFold

#### Récupération des données

In [2]:
def create_lists(path):
    files = glob.glob('../data/news/'+path+'/*.txt')   
    id_articles = []
    articles_contents = []
    for file in files:
        id_article = (file.split("\\")[1]).split('.')[0];
        f = open('../data/news/'+path+'/'+id_article+'.txt','r', encoding='utf-8').read()
        id_articles.append(id_article)
        articles_contents.append(f)
    return id_articles, articles_contents

X_train_ids, X_train_contents = create_lists('training')
X_test_ids, X_test_contents = create_lists('test')


In [3]:
def get_Y_train():
    true_val = []
    indexes = []
    articles = open('../data/labels_training.txt','r').readlines()[1:]
    for i in articles:
        true_val.append(int(i.split(',')[1][:-1]))
        indexes.append(i.split(',')[0])
    return pd.DataFrame(true_val,columns=['fake_news'],index=indexes)

Y_train = get_Y_train().sort_index()

#### Nettoyage des articles

In [4]:
def sentense2cleanTokens(sent):
    sent = sent.lower()
    sent = "".join([x if x.isalpha() else " " for x in sent])
    sent = " ".join(sent.split())
    return sent

In [5]:
X_train = []
for i in X_train_contents:
    X_train.append(sentense2cleanTokens(i))
    
X_test = []
for j in X_test_contents:
    X_test.append(sentense2cleanTokens(j))

In [6]:
tfidf_v = TfidfVectorizer(max_df=0.95, min_df=2, max_features=100000, stop_words='english')
tfidf = tfidf_v.fit_transform(X_train)

#### Partie prédiction

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

D:\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


##### Réseaux de neurones

In [ ]:
model = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1)
np.mean(cross_val_score(model, tfidf, Y_train, cv=10))

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(tfidf, Y_train)

##### Arbres de décision

In [8]:
t = 0
for i in [2, 5, 10, 15, 20, 25, 30, 35]:
    model = DecisionTreeClassifier(max_depth=i)
    pred = np.mean(cross_val_score(model, tfidf, Y_train, cv=10))
    if (pred > t):
        t = pred
        depth = i
print(depth)
print(pred)

15
0.8136842105263158


In [9]:
clf = DecisionTreeClassifier(max_depth=25)
clf.fit(tfidf, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=25,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

##### Gradient boosting

In [10]:
model = GradientBoostingClassifier()
pred = np.mean(cross_val_score(model, tfidf, Y_train, cv=10))
print(pred)
clf_gbc = GradientBoostingClassifier()
clf_gbc.fit(tfidf, Y_train)

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, war

0.8639473684210527


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

#### Partie création du fichier pour push sur kaggle

In [18]:
tfidf_test = tfidf_v.transform(X_test)
#Labels are predicted to the tfidf data
pred_test = clf_gbc.predict(tfidf_test)

In [20]:
fichier = open("pred_test_v2.txt", "a")
fichier.write("doc,class")
for i in range(len(pred_test)):
    fichier.write("\n" + str(X_test_ids[i]) + ',' + str(pred_test[i]))
fichier.close()